In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from dotenv import load_dotenv
import os
from webdav4.fsspec import WebdavFileSystem
from pathlib import Path

## Connect to Sciebo

In [3]:
load_dotenv()
USR = os.getenv('REPORT_USR')
PWD = os.getenv('REPORT_PWD')

## Accessing reports of consultants

In [22]:
fs = WebdavFileSystem("https://uni-bonn.sciebo.de/public.php/webdav", auth=(USR, PWD))
reports = fs.ls('/', detail=False)
assert len(reports) >0, "No reports found!"
consultants = [Path(report).stem for report in reports]
print(f'{len(reports)} reports found and the names of consultants are: {consultants}')

3 reports found and the names of consultants are: ['mohammad', 'nick', 'sangeetha']


## Getting hash of the files

In [25]:
ukeys = [fs.ukey(report) for report in reports]

## Writing names of consultants to a file

In [26]:
os.makedirs('data/', exist_ok=True)
f_consultants = 'data/consultants.txt'
with open(f_consultants, 'w') as f:
    for consultant in zip(consultants, ukeys):
        f.write(f'{consultant[0]}, {consultant[1]}\n')